**So I have my MVP but now I need to add more features to my dataset. The plan is to add the following features:**
1. position
1. height and weight
1. right handed or left
1. went to college or not?
1. Debut age

In [1]:
import pickle
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import math
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from collections import defaultdict 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
%matplotlib inline

In [3]:
# lets load in the current dataframe with all my data
with open('total_players_2000.pkl', 'rb') as handle:
    wow = pickle.load(handle)

In [5]:
wow.head(3)

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,MINORS_BA,MINORS_OPS,MLB_G,MLB_AB,MINORS_R_per_AB,MINORS_SO_per_AB,MINORS_BB_per_AB
0,Evert-Jean 't Hoen,/register/player.fcgi?id=thoen-001eve,485,1576,180,326,38,176,145,458,0.207,0.622,0,0,0.114213,0.290609,0.092005
1,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,0.284,0.833,46,78,0.156048,0.205326,0.093828
2,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,0.259,0.660,79,218,0.150238,0.164149,0.081479


# Web scraping 

In [6]:
#adding more columns for all this data
wow['Position'] = ''
wow['Bats'] = ''
wow['Height'] = ''
wow['Weight'] = ''
wow['Born'] = ''
wow['Debut'] = ''
wow['Draft'] = ''
wow['School'] = ''

In [8]:
#lets write a function that gets all the additional features that I want for each player
def grab_more_features(df):
    """
    Arguments: a dataframe with baseball players and url paths already in there
    Returns: the same dataframe but with more information about them like their Position, if they went to college, 
    the round they were drafted in, batting right-handed or left-handed, when they got to the pros, when they were born,
    their height, and their weight. 
    """
    
    for index,row in enumerate(df.values.tolist()):
        
        #scraping the specific player's stat page
        url = 'https://www.baseball-reference.com' + str(row[1])
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")
        
        #initializing a dictionary to put the data in
        info_dict = defaultdict(str)
        
        #looping through the scraped jumble for exactly the data we want
        for tag in soup.find_all('div', itemtype="https://schema.org/Person"):
            for i,tag2 in enumerate(tag.find_all('p')):
                if tag2.strong is not None:
                    if tag2.strong.text == "Positions:":
                        info_dict["Position"] = tag2.text.replace('\n','').split(':',1)[1].strip().split(' ',1)[0].replace(',','')
                    elif tag2.strong.text == "Position:":
                        info_dict["Position"] = tag2.text.split(':',1)[1].strip()
                    elif tag2.strong.text == "Schools:":
                        info_dict["Schools"] = tag2.text.replace('\n','').split(':',1)[1]
                    elif tag2.strong.text == "School:":
                        info_dict["Schools"] = tag2.text.split(':',1)[1].strip()
                    elif tag2.strong.text == "Bats: ":
                        info_dict["Bats"] = tag2.text.replace('\n','').split(':',1)[1][0:6].strip()
                    elif tag2.strong.text == "Debut:":
                        info_dict["Debut"] = tag2.text.replace('\n','').split(':',1)[1].split('(',1)[0].strip()
                    elif tag2.strong.text == "Born:":
                        info_dict["Born"] = tag2.text.replace('\n','').split(':',1)[1][0:21].strip()
                    elif tag2.strong.text == "Draft":
                        index_num = tag2.text.find('round')
                        info_dict["Draft"] = tag2.text[index_num-5:index_num-3].strip()
                else:
                    for j,tag3 in enumerate(tag2.find_all('span')):
                        if j ==0:
                            info_dict['Height'] = tag3.text
                        if j == 1:
                            info_dict['Weight'] = tag3.text
        
        #assigning the data we grabbed to the specific cells in the dataframe
        df.at[index,'Position'] = info_dict['Position']
        df.at[index,'Bats'] = info_dict['Bats']
        df.at[index,'Height'] = info_dict['Height']
        df.at[index,'Weight'] = info_dict['Weight']
        df.at[index,'Born'] = info_dict['Born']
        df.at[index,'Debut'] = info_dict['Debut']
        if info_dict['Draft']:
            df.at[index,'Draft'] = int(info_dict['Draft'])
        if info_dict['Schools']:
            df.at[index,'School'] = 1
        else:
            df.at[index,'School'] = 0
            
    return df

In [9]:
#running the above function
full_df = grab_more_features(wow) 

In [10]:
full_df.head(3)

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,...,MINORS_SO_per_AB,MINORS_BB_per_AB,Position,Bats,Height,Weight,Born,Debut,Draft,School
0,Evert-Jean 't Hoen,/register/player.fcgi?id=thoen-001eve,485,1576,180,326,38,176,145,458,...,0.290609,0.092005,Shortstop,Right,6-2,190lb,"November 8, 1975",,32,1
1,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,...,0.205326,0.093828,First,Left,6-1,195lb,"July 23, 1974","April 11, 2001",69,1
2,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,...,0.164149,0.081479,Shortstop,Right,5-11,175lb,"August 1, 1974","May 17, 1998",42,1


In [263]:
#saving my progress
with open('full_feature_df.pkl', 'wb') as handle:
    pickle.dump(full_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [469]:
#aaand we're back working
with open('full_feature_df.pkl', 'rb') as handle:
    full_df = pickle.load(handle)

# Data Cleaning (and minor feature engineering)

In [11]:
full_df.Debut

0                         
1           April 11, 2001
2             May 17, 1998
3              May 9, 2001
4       September 10, 1998
               ...        
2382        August 9, 1999
2383     September 3, 1996
2384           May 8, 1994
2385     September 8, 1996
2386     September 1, 1989
Name: Debut, Length: 2387, dtype: object

In [22]:
#changing all the columns with dates to datetime format
full_df.Born = pd.to_datetime(full_df.Born,format="%B %d, %Y",errors='coerce',exact=False)
full_df.Debut = pd.to_datetime(full_df.Debut,format="%B %d, %Y",errors='coerce',exact=False)

In [14]:
#filling in the blank values
full_df.Draft = full_df.Draft.replace('', 0,)
full_df.Draft.astype(int)

0       32
1       69
2       42
3        9
4       10
        ..
2382     7
2383     0
2384    33
2385    40
2386    16
Name: Draft, Length: 2387, dtype: int64

In [16]:
#changing the School column to an int rather than a string
full_df.School = full_df.School.astype('int64')

In [23]:
#making a new feature called Debut Age to figure out the age that each player made it to the MLB
full_df['Debut_Age'] = (full_df.Debut - full_df.Born)/dt.timedelta(days=365)

In [24]:
#filling in the null values
full_df.Born = full_df.Born.fillna(0)
full_df.Debut = full_df.Debut.fillna(0)

In [502]:
with open('full_feature_df.pkl', 'wb') as handle:
    pickle.dump(full_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
#changing height from a string to inches in integers
full_df.Height = full_df.Height.apply(lambda x: (int(x.split('-')[0])*12) + int(x.split('-')[1]))

In [30]:
#cleaning up the categorical variables so we can make them dummies
full_df.loc[full_df.Position=='First', 'Position'] = 'First Baseman'
full_df.loc[full_df.Position=='Second', 'Position'] = 'Second Baseman'
full_df.loc[full_df.Position=='Third', 'Position'] = 'Third Baseman'
full_df.loc[full_df.Position=='Leftfielder', 'Position'] = 'Outfielder'
full_df.loc[full_df.Position=='Rightfielder', 'Position'] = 'Outfielder'
full_df.loc[full_df.Position=='Centerfielder', 'Position'] = 'Outfielder'

In [31]:
#changing the players who either had pinch hitter or designated hitter as their position 
#to their most commonly played field position. We want to eliminate categories with low amounts of players

full_df.at[1086, 'Position'] = 'Outfielder'
full_df.at[2064,'Position'] = 'Third Baseman'
full_df.at[2300,'Position'] = 'Outfielder'
full_df.at[77,'Position'] = 'First Baseman'
full_df.at[588,'Position'] = 'First Baseman'
full_df.at[1465,'Position'] = 'First Baseman'
full_df.at[1968,'Position'] = 'First Baseman'
full_df.at[2018,'Position'] = 'First Baseman'
full_df.at[2088,'Position'] = 'Outfielder'
full_df.at[2124,'Position'] = 'First Baseman'
full_df.at[2184,'Position'] = 'First Baseman'
full_df.at[2244,'Position'] = 'First Baseman'
full_df.at[2335,'Position'] = 'Third Baseman'
full_df.at[2373,'Position'] = 'First Baseman'

In [632]:
#saving progress
with open('full_feature_df.pkl', 'wb') as handle:
    pickle.dump(full_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
#making a new df that is ready to use pd.get_dummies on
dummy_df = full_df.drop(['url_path'],axis=1)
dummy_df.index = dummy_df.player_name
dummy_df = dummy_df.drop(['player_name'],axis=1)
dummy_df = pd.get_dummies(dummy_df)

In [33]:
#changing the names of some columns so they don't have spaces
dummy_df.rename(columns={"Position_Third Baseman":"Position_Third_Baseman", "Position_First Baseman": "Position_First_Baseman", "Position_Second Baseman": "Position_Second_Baseman"}, inplace=True)

In [34]:
#turning weight and height into one feature that uses both of them:bmi
dummy_df['bmi'] = (dummy_df.Weight/2.205)/((dummy_df.Height/39.37)**2)

**Let's save our progress for now**

In [ ]:
with open('dummy_df.pkl', 'wb') as handle:
    pickle.dump(dummy_df, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# Final Data Collection  

**After running some basic models on my data I have one more thing I want to try. Below is some code to figure out how many seasons someone spent in the MLB. The idea is to normalize the MLB_AB column by dividing the ABs by number of seasons**

**At this point, I've also decided to focus my model on predicting MLB ABs only for players who actually made it to the MLB. The model is having trouble with more than half of the observations having 0 as the target variable because the player has no MLB ABs**

In [35]:
#making a new df for only players with MLB ABs
mlb_only = full_df[full_df['MLB_AB'] > 0]
mlb_only.reset_index(drop=True, inplace=True)
mlb_only.loc[:,'MLB_seasons'] = 0
mlb_only.loc[:,'MINORS_seasons'] = 0

/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [37]:
mlb_only.head(3)

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,...,Bats,Height,Weight,Born,Debut,Draft,School,Debut_Age,MLB_seasons,MINORS_seasons
0,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,...,Left,73,195,0,0,69,1,NaN,0,0
1,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,...,Right,71,175,0,0,42,1,NaN,0,0
2,Jamie Burke,/register/player.fcgi?id=burke-002jam,1255,4352,574,1211,54,541,386,572,...,Right,72,195,0,0,9,1,NaN,0,0


In [39]:
def grab_seasons(df):
    """
    Arguments: a dataframe that already has player names and the url path to their stats page on baseball-reference.com
    Returns: returns the dataframe with two more data points: # of seasons in MLB and # of seasons in Minor leagues
    """
    
    #looping through player stat pages and grabbing the HTML
    for index,row in enumerate(df.values.tolist()):
        url = 'https://www.baseball-reference.com' + str(row[1])
        response = requests.get(url)
        page = response.text
        soup = BeautifulSoup(page, "lxml")

        #grabbing number of MLB seasons
        for tag in soup.find_all('tfoot'):
            for tag2 in tag.find_all('tr', class_='mlb'):
                if 'season' in tag2.text:
                    df.at[index,'MLB_seasons'] = tag2.text.split('(')[1].split(' ')[0]

        #grabbing minors seasons
        for tag3 in soup.find_all('tfoot'):
            if tag3.find('tr', class_='minors'):
                for tag4 in tag3.find('tr', class_='minors'):
                    if 'season' in tag4.text:
                        df.at[index,'MINORS_seasons'] = tag4.text.split('(')[1].split(' ')[0]
                        
    return df

In [40]:
#running the above function
mlb_all = grab_seasons(mlb_only)

In [41]:
mlb_all.head(3)

,player_name,url_path,MINORS_G,MINORS_AB,MINORS_R,MINORS_H,MINORS_HR,MINORS_RBI,MINORS_BB,MINORS_SO,...,Bats,Height,Weight,Born,Debut,Draft,School,Debut_Age,MLB_seasons,MINORS_seasons
0,Larry Barnes,/register/player.fcgi?id=barnes002lar,1087,4018,627,1141,152,720,377,825,...,Left,73,195,0,0,69,1,NaN,2,11
1,Justin Baughman,/register/player.fcgi?id=baughm001jus,685,2516,378,652,19,241,205,413,...,Right,71,175,0,0,42,1,NaN,2,7
2,Jamie Burke,/register/player.fcgi?id=burke-002jam,1255,4352,574,1211,54,541,386,572,...,Right,72,195,0,0,9,1,NaN,8,16


In [42]:
#making a new feature
mlb_all['MLB_AB_per_season'] = mlb_all['MLB_AB']/mlb_all['MLB_seasons']

/Users/samir/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [926]:
#saving this new df
with open('mlb_only_full.pkl', 'wb') as handle:
    pickle.dump(mlb_only, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [43]:
#trimming features we already know we are not going to use

#option 1 with MLB AB per season being the target variable
df2 = mlb_only[['MINORS_OPS','MLB_AB_per_season', 'MINORS_R_per_AB',
       'MINORS_SO_per_AB', 'Position', 'Bats', 'Height',
       'Weight', 'Draft', 'School', 'Debut_Age']]

#option 2 with MLB AB total being the target variable
df3 = mlb_only[['MINORS_OPS','MLB_AB', 'MINORS_R_per_AB',
       'MINORS_SO_per_AB', 'Position', 'Bats', 'Height',
       'Weight', 'Draft', 'School', 'Debut_Age']]

In [44]:
#formatting these two new dataframes
df2.index=mlb_only['player_name']
df3.index=mlb_only['player_name']

**I had to redo some of the earlier data cleaning and feature engineering I only applied to dummy_df. So that's why you see some repeated code now**

In [45]:
df2 = pd.get_dummies(df2)
df3 = pd.get_dummies(df3)

In [46]:
df2['bmi'] = (df2.Weight/2.205)/((df2.Height/39.37)**2)
df3['bmi'] = (df3.Weight/2.205)/((df3.Height/39.37)**2)

In [47]:
df2 = df2.drop(['Height', 'Weight'], axis=1)
df3 = df3.drop(['Height', 'Weight'], axis=1)


In [48]:
df2.rename(columns={"Position_Third Baseman":"Position_Third_Baseman", "Position_First Baseman": "Position_First_Baseman", "Position_Second Baseman": "Position_Second_Baseman"}, inplace=True)
df3.rename(columns={"Position_Third Baseman":"Position_Third_Baseman", "Position_First Baseman": "Position_First_Baseman", "Position_Second Baseman": "Position_Second_Baseman"}, inplace=True)

In [49]:
df2.loc['Alfonso Soriano'] = df2.loc['Alfonso Soriano'].fillna(0.791)
df3.loc['Alfonso Soriano'] = df3.loc['Alfonso Soriano'].fillna(0.791)

**Ok done! Now time to save the data and proceed to EDA, Regularization and Validation**

In [987]:
with open('per_season_mlb_eda_ready.pkl', 'wb') as handle:
    pickle.dump(df2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [988]:
with open('per_ab_mlb_eda_ready.pkl', 'wb') as handle:
    pickle.dump(df3, handle, protocol=pickle.HIGHEST_PROTOCOL)